# 간단한 RAG에서의 문맥적 청크 헤더 (CCH)

검색 증강 생성(RAG)은 응답을 생성하기 전에 관련 외부 지식을 검색하여 언어 모델의 사실적 정확성을 향상시킵니다. 그러나 표준적인 청킹 방식은 종종 중요한 문맥을 잃어버려 검색 효율성을 떨어뜨립니다.

문맥적 청크 헤더(Contextual Chunk Headers, CCH)는 각 청크를 임베딩하기 전에 상위 수준의 문맥(예: 문서 제목 또는 섹션 헤더)을 각 청크 앞에 추가하여 RAG를 향상시킵니다. 이는 검색 품질을 향상시키고 문맥에서 벗어난 응답을 방지합니다.
즉, 각 텍스트 조각(청크)에 해당 조각이 어떤 내용에 관한 것인지를 요약하는 헤더를 붙여줌으로써, 검색 시 더 정확한 정보를 찾고, 언어 모델이 답변을 생성할 때도 해당 조각의 문맥을 더 잘 이해하도록 돕는 기법입니다.

## 이 노트북의 단계:

1. **데이터 수집**: 텍스트 데이터를 로드하고 전처리합니다.
2. **문맥적 헤더를 사용한 청킹**: 섹션 제목을 추출하여 청크 앞에 추가합니다.
3. **임베딩 생성**: 문맥이 강화된 청크를 숫자 표현으로 변환합니다.
4. **시맨틱 검색**: 사용자 질의를 기반으로 관련 청크를 검색합니다.
5. **응답 생성**: 검색된 텍스트로부터 응답을 생성하기 위해 언어 모델을 사용합니다.
6. **평가**: 점수 시스템을 사용하여 응답 정확도를 평가합니다.

## 환경 설정
필요한 라이브러리를 가져오는 것으로 시작합니다.

In [1]:
import os
import numpy as np
import json
from openai import OpenAI
import fitz  # PyMuPDF
from tqdm import tqdm # 진행률 표시

## 텍스트 추출 및 섹션 헤더 식별
PDF에서 텍스트를 추출하는 동시에 섹션 제목(청크의 잠재적 헤더)을 식별합니다.

In [2]:
def extract_text_from_pdf(pdf_path):
    """
    PDF 파일에서 텍스트를 추출하고 처음 `num_chars`개의 문자를 출력합니다.

    Args:
    pdf_path (str): PDF 파일 경로.

    Returns:
    str: PDF에서 추출된 텍스트.
    """
    # PDF 파일 열기
    mypdf = fitz.open(pdf_path)
    all_text = ""  # 추출된 텍스트를 저장할 빈 문자열 초기화

    # PDF의 각 페이지 반복
    for page_num in range(mypdf.page_count):
        page = mypdf[page_num]  # 페이지 가져오기
        text = page.get_text("text")  # 페이지에서 텍스트 추출
        all_text += text  # 추출된 텍스트를 all_text 문자열에 추가

    return all_text  # 추출된 텍스트 반환

## OpenAI API 클라이언트 설정
임베딩과 응답을 생성하기 위해 OpenAI 클라이언트를 초기화합니다.

In [ ]:
# 기본 URL과 API 키로 OpenAI 클라이언트 초기화
client = OpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.getenv("OPENAI_API_KEY")  # 환경 변수에서 API 키 검색
)

## 문맥적 헤더를 사용한 텍스트 청킹
검색 성능을 향상시키기 위해 LLM 모델을 사용하여 각 청크에 대한 설명적인 헤더를 생성합니다.
각 청크의 내용을 요약하는 헤더를 생성하여 청크의 주요 내용을 명확히 하고, 검색 시 이 헤더 정보도 함께 활용하여 관련성을 높입니다.

In [4]:
def generate_chunk_header(chunk, model="meta-llama/Llama-3.2-3B-Instruct"):
    """
    LLM을 사용하여 주어진 텍스트 청크에 대한 제목/헤더를 생성합니다.

    Args:
    chunk (str): 헤더로 요약할 텍스트 청크.
    model (str): 헤더 생성에 사용할 모델. 기본값은 "meta-llama/Llama-3.2-3B-Instruct"입니다.

    Returns:
    str: 생성된 헤더/제목.
    """
    # AI의 행동을 안내하는 시스템 프롬프트 정의
    system_prompt = "주어진 텍스트에 대해 간결하고 유익한 제목을 생성하십시오."
    
    # 시스템 프롬프트와 텍스트 청크를 기반으로 AI 모델로부터 응답 생성
    response = client.chat.completions.create(
        model=model,
        temperature=0, # 결정론적 결과를 위해 온도를 0으로 설정
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": chunk}
        ]
    )

    # 생성된 헤더/제목 반환 (앞뒤 공백 제거)
    return response.choices[0].message.content.strip()

In [5]:
def chunk_text_with_headers(text, n, overlap):
    """
    텍스트를 더 작은 세그먼트로 청킹하고 헤더를 생성합니다.

    Args:
    text (str): 청킹할 전체 텍스트.
    n (int): 문자 단위의 청크 크기.
    overlap (int): 청크 간 중첩되는 문자 수.

    Returns:
    List[dict]: 'header'와 'text' 키를 가진 딕셔너리 목록.
    """
    chunks = []  # 청크를 저장할 빈 리스트 초기화

    # 지정된 청크 크기와 중첩으로 텍스트 반복
    for i in range(0, len(text), n - overlap):
        chunk_text_content = text[i:i + n]  # 텍스트 청크 추출
        header = generate_chunk_header(chunk_text_content)  # LLM을 사용하여 청크에 대한 헤더 생성
        chunks.append({"header": header, "text": chunk_text_content})  # 헤더와 청크를 리스트에 추가

    return chunks  # 헤더가 있는 청크 목록 반환

## PDF 파일에서 텍스트 추출 및 청킹
이제 PDF를 로드하고, 텍스트를 추출한 다음, 청크로 분할합니다.

In [6]:
# PDF 파일 경로 정의
pdf_path = "data/AI_Information.pdf"

# PDF 파일에서 텍스트 추출
extracted_text = extract_text_from_pdf(pdf_path)

# 헤더와 함께 추출된 텍스트 청킹
# 청크 크기는 1000자, 중첩은 200자로 사용합니다.
text_chunks = chunk_text_with_headers(extracted_text, 1000, 200)

# 생성된 헤더와 함께 샘플 청크 출력
print("샘플 청크:")
print("헤더:", text_chunks[0]['header'])
print("내용:", text_chunks[0]['text'])

Sample Chunk:
Header: "Introduction to Artificial Intelligence: Understanding the Foundations and Evolution"
Content: Understanding Artificial Intelligence 
Chapter 1: Introduction to Artificial Intelligence 
Artificial intelligence (AI) refers to the ability of a digital computer or computer-controlled robot 
to perform tasks commonly associated with intelligent beings. The term is frequently applied to 
the project of developing systems endowed with the intellectual processes characteristic of 
humans, such as the ability to reason, discover meaning, generalize, or learn from past 
experience. Over the past few decades, advancements in computing power and data availability 
have significantly accelerated the development and deployment of AI. 
Historical Context 
The idea of artificial intelligence has existed for centuries, often depicted in myths and fiction. 
However, the formal field of AI research began in the mid-20th century. The Dartmouth Workshop 
in 1956 is widely considered

## 헤더 및 텍스트에 대한 임베딩 생성
검색 정확도를 향상시키기 위해 헤더와 텍스트 모두에 대한 임베딩을 생성합니다.
청크의 내용뿐만 아니라 생성된 헤더에 대해서도 별도의 임베딩을 만들어, 검색 시 이 두 가지 정보를 함께 활용합니다. 이렇게 하면 질의와 청크 내용 간의 유사도뿐만 아니라 질의와 청크 헤더 간의 유사도도 고려할 수 있어 더 정확한 검색이 가능해집니다.

In [7]:
def create_embeddings(text, model="BAAI/bge-en-icl"):
    """
    주어진 텍스트에 대한 임베딩을 생성합니다.

    Args:
    text (str): 임베딩할 입력 텍스트.
    model (str): 사용할 임베딩 모델. 기본값은 "BAAI/bge-en-icl"입니다.

    Returns:
    dict: 입력 텍스트에 대한 임베딩을 포함하는 응답.
    """
    # 지정된 모델과 입력 텍스트를 사용하여 임베딩 생성
    response = client.embeddings.create(
        model=model,
        input=text
    )
    # 응답에서 임베딩 반환
    return response.data[0].embedding

In [8]:
# 각 청크에 대한 임베딩 생성
embeddings = []  # 임베딩을 저장할 빈 리스트 초기화

# 진행률 표시줄과 함께 각 텍스트 청크 반복
for chunk in tqdm(text_chunks, desc="임베딩 생성 중"):
    # 청크 텍스트에 대한 임베딩 생성
    text_embedding = create_embeddings(chunk["text"])
    # 청크 헤더에 대한 임베딩 생성
    header_embedding = create_embeddings(chunk["header"])
    # 청크 헤더, 텍스트 및 해당 임베딩을 리스트에 추가
    embeddings.append({"header": chunk["header"], "text": chunk["text"], "embedding": text_embedding, "header_embedding": header_embedding})

Generating embeddings: 100%|██████████| 42/42 [02:56<00:00,  4.21s/it]


## 시맨틱 검색 수행
사용자 질의에 가장 관련성 높은 텍스트 청크를 찾기 위해 코사인 유사도를 구현합니다.
검색 시에는 질의어 임베딩과 각 청크의 (1) 텍스트 임베딩 간의 유사도, (2) 헤더 임베딩 간의 유사도를 모두 계산하여 종합적으로 관련성을 판단합니다. 예를 들어, 두 유사도의 평균을 사용할 수 있습니다.

In [9]:
def cosine_similarity(vec1, vec2):
    """
    두 벡터 간의 코사인 유사도를 계산합니다.

    Args:
    vec1 (np.ndarray): 첫 번째 벡터.
    vec2 (np.ndarray): 두 번째 벡터.

    Returns:
    float: 코사인 유사도 점수.
    """
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [10]:
def semantic_search(query, chunks, k=5):
    """
    질의를 기반으로 가장 관련성 높은 청크를 검색합니다.

    Args:
    query (str): 사용자 질의.
    chunks (List[dict]): 임베딩이 포함된 텍스트 청크 목록.
    k (int): 상위 결과 수.

    Returns:
    List[dict]: 상위 k개의 가장 관련성 높은 청크.
    """
    # 질의에 대한 임베딩 생성
    query_embedding = create_embeddings(query)

    similarities = []  # 유사도 점수를 저장할 리스트 초기화
    
    # 각 청크를 반복하여 유사도 점수 계산
    for chunk_data in chunks: # 변수명을 chunk에서 chunk_data로 변경하여 명확성 향상
        # 질의 임베딩과 청크 텍스트 임베딩 간의 코사인 유사도 계산
        sim_text = cosine_similarity(np.array(query_embedding), np.array(chunk_data["embedding"]))
        # 질의 임베딩과 청크 헤더 임베딩 간의 코사인 유사도 계산
        sim_header = cosine_similarity(np.array(query_embedding), np.array(chunk_data["header_embedding"]))
        # 평균 유사도 점수 계산 (텍스트 유사도와 헤더 유사도의 가중 평균도 고려 가능)
        avg_similarity = (sim_text + sim_header) / 2
        # 청크와 해당 평균 유사도 점수를 리스트에 추가
        similarities.append((chunk_data, avg_similarity))

    # 유사도 점수를 기준으로 청크를 내림차순 정렬
    similarities.sort(key=lambda x: x[1], reverse=True)
    # 상위 k개의 가장 관련성 높은 청크 반환
    return [x[0] for x in similarities[:k]]

## 추출된 청크에 대한 질의 실행

In [11]:
# 검증 데이터 로드
with open('data/val.json') as f:
    data = json.load(f)

query = data[0]['question']

# 상위 2개의 가장 관련성 높은 텍스트 청크 검색
top_chunks = semantic_search(query, embeddings, k=2)

# 결과 출력
print("질의:", query)
for i, chunk_info in enumerate(top_chunks): # 변수명을 chunk에서 chunk_info로 변경
    print(f"헤더 {i+1}: {chunk_info['header']}")
    print(f"내용:\n{chunk_info['text']}\n")

Query: What is 'Explainable AI' and why is it considered important?
Header 1: "Building Trust in AI: Addressing Transparency, Explainability, and Accountability"
Content:
systems. Explainable AI (XAI) 
techniques aim to make AI decisions more understandable, enabling users to assess their 
fairness and accuracy. 
Privacy and Data Protection 
AI systems often rely on large amounts of data, raising concerns about privacy and data 
protection. Ensuring responsible data handling, implementing privacy-preserving techniques, 
and complying with data protection regulations are crucial. 
Accountability and Responsibility 
Establishing accountability and responsibility for AI systems is essential for addressing potential 
harms and ensuring ethical behavior. This includes defining roles and responsibilities for 
developers, deployers, and users of AI systems. 
Chapter 20: Building Trust in AI 
Transparency and Explainability 
Transparency and explainability are key to building trust in AI. Maki

## 검색된 청크를 기반으로 응답 생성

In [12]:
# AI 어시스턴트에 대한 시스템 프롬프트 정의
system_prompt = "당신은 주어진 문맥에 기반하여 엄격하게 답변하는 AI 어시스턴트입니다. 제공된 문맥에서 직접적으로 답변을 도출할 수 없는 경우, '답변하기에 충분한 정보가 없습니다.'라고 응답하십시오."

def generate_response(system_prompt, user_message, model="meta-llama/Llama-3.2-3B-Instruct"):
    """
    시스템 프롬프트와 사용자 메시지를 기반으로 AI 모델로부터 응답을 생성합니다.

    Args:
    system_prompt (str): AI의 행동을 안내하는 시스템 프롬프트.
    user_message (str): 사용자의 메시지 또는 질의.
    model (str): 응답 생성에 사용할 모델. 기본값은 "meta-llama/Llama-3.2-3B-Instruct"입니다. (원문: "meta-llama/Llama-2-7B-chat-hf")

    Returns:
    dict: AI 모델의 응답.
    """
    response = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ]
    )
    return response

# 상위 청크를 기반으로 사용자 프롬프트 생성 (헤더와 내용을 함께 전달)
user_prompt = "\n".join([f"헤더: {chunk_data['header']}\n내용:\n{chunk_data['text']}" for chunk_data in top_chunks])
user_prompt = f"{user_prompt}\n질문: {query}"

# AI 응답 생성
ai_response = generate_response(system_prompt, user_prompt)

## AI 응답 평가
AI 응답을 예상 답변과 비교하고 점수를 매깁니다.

In [13]:
# 평가 시스템 프롬프트 정의
evaluate_system_prompt = """당신은 지능형 평가 시스템입니다. 
제공된 문맥을 기반으로 AI 어시스턴트의 응답을 평가하십시오. 
- 응답이 실제 답변과 매우 유사하면 1점을 부여하십시오. 
- 응답이 부분적으로 정확하면 0.5점을 부여하십시오. 
- 응답이 부정확하면 0점을 부여하십시오.
점수(0, 0.5 또는 1)만 반환하십시오."""

# 검증 데이터에서 정답 추출
true_answer = data[0]['ideal_answer']

# 평가 프롬프트 구성
evaluation_prompt = f"""
사용자 질의: {query}
AI 응답: {ai_response.choices[0].message.content} # AI 응답 내용에 접근
실제 답변: {true_answer}
{evaluate_system_prompt}
"""

# 평가 점수 생성
evaluation_response = generate_response(evaluate_system_prompt, evaluation_prompt)

# 평가 점수 출력
print("평가 점수:", evaluation_response.choices[0].message.content)

Evaluation Score: 0.5
